#DATASCI W261: Machine Learning at Scale

#This notebook shows a Hadoop MapReduce job of WordCount.

##Data

In [1]:
%%writefile wordcount.txt
hello hi hi hallo
bonjour hola hi ciao
nihao konnichiwa ola
hola nihao hello

Overwriting wordcount.txt


##Mapper

In [2]:
%%writefile mapper.py
#!/usr/bin/python
import sys
# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print '%s\t%s' % (word, 1)

Overwriting mapper.py


##Reducer

In [3]:
%%writefile reducer.py
#!/usr/bin/python
from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


##Test Code

In [4]:
!echo "foo foo quux labs foo bar quux" | python mapper.py | sort -k1,1 | python reducer.py

bar	1
foo	3
labs	1
quux	2


## Run wordcount in hadoop

###start yarn and hdfs

In [2]:
!/usr/local/Cellar/hadoop/2.6.0/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.6.0/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/yarn-liang-resourcemanager-ldai.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/yarn-liang-nodemanager-ldai.out
15/08/21 06:23:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/hadoop-liang-namenode-ldai.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/hadoop-liang-datanode-ldai.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.6.0/libexec/logs/hadoop-liang-secondarynamenode-ldai.out
15/08/21 06:23:43 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


###create a folder 

In [3]:
!hdfs dfs -mkdir -p /user/liang

15/08/21 06:23:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


###upload wordcount.txt to hdfs

In [4]:
!hdfs dfs -put wordcount.txt /user/liang

15/08/21 06:23:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
put: `/user/liang/wordcount.txt': File exists


### Hadoop streaming command

<pre>
hadoop jar hadoopstreamingjarfile \
    -D stream.num.map.output.key.fields=n \
    -mapper mapperfile \
    -reducer reducerfile \
    -input inputfile \
    -output outputfile</pre>

hadoop streaming jar file can be found in your hadoop folder or downloaded from
http://mvnrepository.com/artifact/org.apache.hadoop/hadoop-streaming/2.6.0

In [7]:
!hadoop jar hadoop-*streaming*.jar -mapper mapper.py -reducer reducer.py -input wordcount.txt -output wordcountOutput

15/08/21 06:24:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/08/21 06:24:38 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
15/08/21 06:24:38 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
15/08/21 06:24:38 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
15/08/21 06:24:39 INFO mapred.FileInputFormat: Total input paths to process : 1
15/08/21 06:24:39 INFO mapreduce.JobSubmitter: number of splits:1
15/08/21 06:24:39 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1633678757_0001
15/08/21 06:24:40 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
15/08/21 06:24:40 INFO mapred.LocalJobRunner: OutputCommitter set in config null
15/08/21 06:24:40 INFO mapreduce.Job: Running job: job_local1633678757_0001
15/08/21 06:24:4

###show the results

In [8]:
!hdfs dfs -cat wordcountOutput/part-00000

15/08/21 06:28:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
bonjour	1
ciao	1
hallo	1
hello	2
hi	3
hola	2
konnichiwa	1
nihao	2
ola	1


###stop yarn and hdfs

In [9]:
!/usr/local/Cellar/hadoop/2.6.0/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.6.0/sbin/stop-dfs.sh

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
15/08/21 06:28:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
15/08/21 06:29:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
